In [1]:
# PyAlex and Data Creation imports
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
import pyalex, os, csv
import requests, time

pyalex.config.email = "b.cliff@gwmail.gwu.edu"
from itertools import chain

In [2]:
# Data Validation imports
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, List, Dict

#### Configuring Output Files

In [3]:
data_folder_loc = './data'

In [4]:
WORKS_FILE = f'{data_folder_loc}/works.csv'
AUTHORS_FILE = f'{data_folder_loc}/authors.csv'
TOPICS_FILE = f'{data_folder_loc}/topics.csv'
PUBLISHERS_FILE = f'{data_folder_loc}/publishers.csv'
FUNDERS_FILE = f'{data_folder_loc}/funders.csv'
WORK_AUTH_FILE = f'{data_folder_loc}/work_auth_edges.csv'
WORK_TOPIC_FILE = f'{data_folder_loc}/work_topic_edges.csv'
INSTITUTIONS_FILE = f'{data_folder_loc}/institutions.csv'
SOURCES_FILE = f'{data_folder_loc}/sources.csv'


In [5]:
AWARD_INVESTIGATOR_FILE = f'{data_folder_loc}/award_investigator_edges.csv'
AWARDS_FILE = f'{data_folder_loc}/awards.csv'

In [6]:
# Removing any old/vestige files
# Add AWARDS_FILE to your file removal section
for path in [WORKS_FILE, AUTHORS_FILE, TOPICS_FILE, WORK_AUTH_FILE,
             WORK_TOPIC_FILE, INSTITUTIONS_FILE, SOURCES_FILE, AWARDS_FILE,
             PUBLISHERS_FILE, FUNDERS_FILE, AWARD_INVESTIGATOR_FILE]:
    if os.path.exists(path):
        os.remove(path)


In [7]:
works_file = open(WORKS_FILE, 'w', newline='',encoding='utf-8')
authors_file = open(AUTHORS_FILE, 'w', newline='', encoding='utf-8')
topics_file = open(TOPICS_FILE, 'w', newline='', encoding='utf-8')
institutions_file = open(INSTITUTIONS_FILE, 'w', newline='', encoding='utf-8')
publishers_file = open(PUBLISHERS_FILE, 'w', newline='', encoding='utf-8')
funders_file = open(FUNDERS_FILE, 'w', newline='', encoding='utf-8')
sources_file = open(SOURCES_FILE, 'w', newline='', encoding='utf-8')
awards_file = open(AWARDS_FILE, 'w', newline='', encoding='utf-8')

In [8]:
# Edge files for ingestion
wt_edge_file = open(WORK_TOPIC_FILE, 'w', newline='', encoding='utf-8')
wa_edge_file = open(WORK_AUTH_FILE, 'w', newline='', encoding='utf-8')
award_investigator_file = open(AWARD_INVESTIGATOR_FILE, 'w', newline='', encoding='utf-8')

In [9]:
# Creating the headers
works_writer = csv.writer(works_file)
authors_writer = csv.writer(authors_file)
topics_writer = csv.writer(topics_file)
institutions_writer = csv.writer(institutions_file)
publishers_writer = csv.writer(publishers_file)
funders_writer = csv.writer(funders_file)
sources_writer = csv.writer(sources_file)
awards_writer = csv.writer(awards_file)
award_investigator_writer = csv.writer(award_investigator_file)

In [10]:
wt_edges_writer = csv.writer(wt_edge_file)
wa_edges_writer = csv.writer(wa_edge_file)

In [11]:
# Writing the headers to the csv files
works_writer.writerow(['id', 'doi', 'title', 'publication_date', 'type', 'related_works', 'corresponding_author_ids'])
authors_writer.writerow(['id', 'name', 'works_count', 'cited_by_count', 'affiliations', 'topics', 'counts_by_year'])
institutions_writer.writerow(['id', 'display_name', 'ror', 'country_code', 'type', 'works_count', 'cited_by_count', 'homepage_url', 'is_super_system', 'created_date', 'updated_date'])
publishers_writer.writerow(['id', 'display_name', 'country_codes', 'hierarchy_level', 'parent_publisher', 'works_count', 'cited_by_count', 'created_date', 'updated_date'])
funders_writer.writerow(['id', 'display_name', 'description', 'country_code', 'awards_count', 'works_count', 'cited_by_count', 'homepage_url', 'created_date', 'updated_date'])
sources_writer.writerow(['id', 'display_name', 'host_organization_name', 'works_count', 'cited_by_count', 'first_publication_year', 'last_publication_year', 'created_date', 'updated_date'])
wa_edges_writer.writerow(['work_id', 'author_id', 'author_position', 'is_corresponding'])
wt_edges_writer.writerow(['work_id', 'topic_id', 'topic_name', 'score'])

35

In [12]:
topics_writer.writerow(['id', 'topic_name', 'description', 'keywords', 
                        'subfield', 'field', 'domain', 'works_count',
                        'cited_by_count' ,'updated_date' ,'created_date'])

# Update your CSV headers
awards_writer.writerow(['id', 'display_name', 'description', 'funder_award_id', 'funder', 
                       'funded_outputs_count', 'amount', 'currency', 'funding_type', 
                       'funder_scheme', 'start_date', 'end_date', 'start_year', 'end_year',
                       'landing_page_url', 'doi', 'provenance', 'created_date', 'updated_date'])

award_investigator_writer.writerow(['award_id', 'investigator_orcid', 'investigator_given_name',
                                   'investigator_family_name', 'investigator_full_name', 'role',
                                   'role_start', 'affiliation_name', 'affiliation_country'])


154

#### Creating Pydantic Models for Validation

In [13]:
class WorksModel(BaseModel):
    id: str
    doi: Optional[str] = None
    title: str
    publication_date: str
    type: Optional[str] = None
    related_works: List[str] = None
    corresponding_author_ids: List[str] = None

In [14]:
class AuthorsModel(BaseModel):
    id: str
    display_name: str
    works_count: int
    cited_by_count: int
    affiliations: List[dict] = None
    topics: List[dict] = None
    counts_by_year: List[dict]

In [15]:
class TopicsModel(BaseModel):
    id: str
    display_name: str
    description: str
    keywords: List[str] = None
    subfield_name: Optional[str] = None
    field_name: Optional[str] = None
    domain_name: Optional[str] = None
    works_count: int
    cited_by_count: int
    updated_date: str
    created_date: str

In [16]:
class InstitutionsModel(BaseModel):
    id: str
    display_name: str
    ror: Optional[str] = None
    country_code: Optional[str] = None
    type: Optional[str] = None
    works_count: int
    cited_by_count: int
    homepage_url: Optional[str] = None
    is_super_system: bool
    created_date: str
    updated_date: str

In [17]:
class PublishersModel(BaseModel):
    id: str
    display_name: str
    country_code: List[str] = None
    # hierarchy_level: int
    parent_publisher: Optional[str] = None
    works_count: int
    cited_by_count: int
    created_date: str
    updated_date: str

In [18]:
class FundersModel(BaseModel):
    id: str
    display_name: str
    description: Optional[str] = None
    country_code: Optional[str] = None
    awards_count: int
    works_count: int
    cited_by_count: int
    homepage_url: Optional[str] = None
    created_date: str
    updated_date: str

In [19]:
class SourcesModel(BaseModel):
    id: str
    display_name: str
    host_organization_name: Optional[str] = None  # Can be None
    works_count: int
    cited_by_count: int
    first_publication_year: Optional[int] = None  # Can be None
    last_publication_year: Optional[int] = None   # Can be None
    created_date: str
    updated_date: str


In [20]:
class AwardsModel(BaseModel):
    id: str
    display_name: Optional[str] = None
    description: Optional[str] = None
    funder_award_id: Optional[str] = None
    funder: Optional[str] = None  # Will be cleaned to just ID
    funded_outputs_count: Optional[int] = None
    amount: Optional[float] = None
    currency: Optional[str] = None
    funding_type: Optional[str] = None
    funder_scheme: Optional[str] = None
    start_date: Optional[str] = None
    end_date: Optional[str] = None
    start_year: Optional[int] = None
    end_year: Optional[int] = None
    landing_page_url: Optional[str] = None
    doi: Optional[str] = None
    provenance: Optional[str] = None
    created_date: str
    updated_date: str

class AwardInvestigatorEdge(BaseModel):
    award_id: str
    investigator_orcid: Optional[str] = None
    investigator_given_name: Optional[str] = None
    investigator_family_name: Optional[str] = None
    investigator_full_name: str
    role: str  # "lead", "co_lead", "investigator"
    role_start: Optional[str] = None
    affiliation_name: Optional[str] = None
    affiliation_country: Optional[str] = None


### Data retrieval from OpenAlex

In [21]:
oa_replace = 'https://openalex.org/'

In [22]:
works_pager = Works().select(['id', 'doi', 'title',
'publication_date', 'type', 'related_works', 'corresponding_author_ids']).paginate(per_page=200)

In [23]:
authors_pager = Authors().select(['id', 'orcid', 'display_name',
                                  'works_count', 'cited_by_count',
                                  'affiliations', 'topics', 'counts_by_year']).paginate(per_page=200)

In [24]:
topics_pager = Topics().select(['id', 'display_name', 'description', 'keywords',
                                'subfield', 'field', 'domain', 'works_count',
                                'cited_by_count', 'updated_date', 'created_date']).paginate(per_page=200)

In [25]:
institutions_pager = Institutions().select(['id', 'display_name', 'ror', 'country_code', 'type',
                                           'works_count', 'cited_by_count', 'homepage_url',
                                           'is_super_system', 'created_date', 'updated_date']).paginate(per_page=200)

In [26]:
publishers_pager = Publishers().select(['id', 'display_name', 'country_codes', 'hierarchy_level',
                                       'parent_publisher', 'works_count', 'cited_by_count',
                                       'created_date', 'updated_date']).paginate(per_page=200)

In [27]:
funders_pager = Funders().select(['id', 'display_name', 'description', 'country_code',
                                 'awards_count', 'works_count', 'cited_by_count',
                                 'homepage_url', 'created_date', 'updated_date']).paginate(per_page=200)

In [28]:
sources_pager = Sources().select(['id', 'display_name', 'host_organization_name', 'works_count', 
'cited_by_count', 'first_publication_year', 
'last_publication_year', 'created_date', 'updated_date']).paginate(per_page=200)

## Adding Awards from OpenAlex Directly

In [29]:
def fetch_awards_data():
    """Fetch awards data with lead investigator filter"""
    url = "https://api.openalex.org/awards"
    page = 1
    
    while True:
        params = {
            'per-page': 200,
            'page': page,
            'mailto': 'b.cliff@gwmail.gwu.edu',
            'filter': 'lead_investigator.given_name:!null,amount:>0,end_year:>2019'  # Your working filter
        }
        
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"API request failed: {response.status_code}")
            break
            
        data = response.json()
        results = data.get('results', [])
        
        if not results:
            print("No more results from API")
            break
            
        for award in results:
            yield award
            
        page += 1
        time.sleep(0.1)
        
        # Optional: Progress reporting
        if page % 25 == 0:  # Every 5,000 records
            print(f"Fetched {page * 200} awards so far...")


In [30]:
def extract_investigator_info(investigator_data: dict) -> dict:
    """Extract investigator information from API response"""
    if not investigator_data:
        return {}
    
    # Extract affiliation info
    affiliation = investigator_data.get('affiliation', {})
    
    return {
        'given_name': investigator_data.get('given_name'),
        'family_name': investigator_data.get('family_name'),
        'orcid': investigator_data.get('orcid'),
        'role_start': investigator_data.get('role_start'),
        'affiliation_name': affiliation.get('name') if affiliation else None,
        'affiliation_country': affiliation.get('country') if affiliation else None
    }

def create_investigator_name(investigator_info: dict) -> str:
    """Create a full name for investigator"""
    given = investigator_info.get('given_name', '') or ''
    family = investigator_info.get('family_name', '') or ''
    full_name = f"{given} {family}".strip()
    return full_name if full_name else "Unknown"

def write_investigator_edge(award_id: str, investigator_info: dict, role: str, writer):
    """Write investigator relationship to CSV"""
    try:
        full_name = create_investigator_name(investigator_info)
        
        edge = AwardInvestigatorEdge(
            award_id=award_id,
            investigator_orcid=investigator_info.get('orcid'),
            investigator_given_name=investigator_info.get('given_name'),
            investigator_family_name=investigator_info.get('family_name'),
            investigator_full_name=full_name,
            role=role,
            role_start=investigator_info.get('role_start'),
            affiliation_name=investigator_info.get('affiliation_name'),
            affiliation_country=investigator_info.get('affiliation_country')
        )
        
        writer.writerow([
            edge.award_id,
            edge.investigator_orcid,
            edge.investigator_given_name,
            edge.investigator_family_name,
            edge.investigator_full_name,
            edge.role,
            edge.role_start,
            edge.affiliation_name,
            edge.affiliation_country
        ])
    except ValidationError as e:
        print(f"Skipping invalid investigator edge for award {award_id}: {e}")


In [31]:
def process_single_award(raw):
    """Process a single award, return True if successfully written to CSV"""
    try:
        award_id = raw['id'].replace(oa_replace, '')
        
        # Extract investigator data (we know lead_investigator exists due to filter)
        lead_investigator = raw.pop('lead_investigator', None)
        co_lead_investigator = raw.pop('co_lead_investigator', None)
        investigators = raw.pop('investigators', None)  # This might be None or []
        
        # Clean other fields
        raw['id'] = award_id
        
        # Clean funder field (it's a dict with id and display_name)
        if raw.get('funder') and isinstance(raw['funder'], dict):
            funder_id = raw['funder'].get('id')
            raw['funder'] = funder_id.replace(oa_replace, '') if funder_id else None
        
        # Handle missing display_name
        if not raw.get('display_name'):
            raw['display_name'] = f"Award {award_id}"
        
        # Skip records with critical missing data
        if not raw.get('created_date') or not raw.get('updated_date'):
            return False
        
        # Validate and write award data
        a = AwardsModel(**raw)
        awards_writer.writerow([
            a.id, a.display_name, a.description, a.funder_award_id, a.funder,
            a.funded_outputs_count, a.amount, a.currency, a.funding_type,
            a.funder_scheme, a.start_date, a.end_date, a.start_year, a.end_year,
            a.landing_page_url, a.doi, a.provenance, a.created_date, a.updated_date
        ])
        
        # Process lead investigator (should always exist due to filter)
        if lead_investigator:
            lead_info = extract_investigator_info(lead_investigator)
            write_investigator_edge(award_id, lead_info, "lead", award_investigator_writer)
        
        # Process co-lead investigator
        if co_lead_investigator:
            co_lead_info = extract_investigator_info(co_lead_investigator)
            write_investigator_edge(award_id, co_lead_info, "co_lead", award_investigator_writer)
        
        # Process other investigators (if they exist)
        if investigators and isinstance(investigators, list):
            for investigator in investigators:
                inv_info = extract_investigator_info(investigator)
                write_investigator_edge(award_id, inv_info, "investigator", award_investigator_writer)
        
        return True
        
    except Exception as e:
        print(f"Error processing award: {e}")
        return False


def process_awards_until_target(target_count=10000):
    """Process awards until we've written target_count to CSV"""
    awards_written = 0
    total_fetched = 0
    skipped_count = 0
    
    print(f"Processing Awards until {target_count} are written to CSV...")
    
    for raw in fetch_awards_data():
        total_fetched += 1
        
        # Check if we've reached our target
        if awards_written >= target_count:
            print(f"Target reached! Stopping after writing {awards_written} awards.")
            break
        
        # Process the award
        success = process_single_award(raw)
        
        if success:
            awards_written += 1
            # Progress reporting
            if awards_written % 500 == 0:
                print(f"Progress: {awards_written}/{target_count} awards written (fetched {total_fetched} total)")
        else:
            skipped_count += 1
    
    return awards_written, total_fetched, skipped_count

# Use it
print("Starting award processing...")
awards_written, total_fetched, skipped = process_awards_until_target(10000)

awards_file.close()
award_investigator_file.close()

print(f"Awards processing complete!")
print(f"Final results: {awards_written} written, {total_fetched} fetched, {skipped} skipped")
print(f"Success rate: {(awards_written/total_fetched)*100:.1f}%")


Starting award processing...
Processing Awards until 10000 are written to CSV...
Progress: 500/10000 awards written (fetched 500 total)
Progress: 1000/10000 awards written (fetched 1000 total)
Progress: 1500/10000 awards written (fetched 1500 total)
Progress: 2000/10000 awards written (fetched 2000 total)
Progress: 2500/10000 awards written (fetched 2500 total)
Progress: 3000/10000 awards written (fetched 3000 total)
Progress: 3500/10000 awards written (fetched 3500 total)
Progress: 4000/10000 awards written (fetched 4000 total)
Progress: 4500/10000 awards written (fetched 4500 total)
Fetched 5000 awards so far...
Progress: 5000/10000 awards written (fetched 5000 total)
Progress: 5500/10000 awards written (fetched 5500 total)
Progress: 6000/10000 awards written (fetched 6000 total)
Progress: 6500/10000 awards written (fetched 6500 total)
Progress: 7000/10000 awards written (fetched 7000 total)
Progress: 7500/10000 awards written (fetched 7500 total)
Progress: 8000/10000 awards written 

#### Data Cleaning

In [33]:
for page in sources_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            # Clean host organization ID if present
            if raw.get('host_organization_name'):
                raw['host_organization_name'] = raw['host_organization_name'].replace(oa_replace, '')
            s = SourcesModel(**raw)
            sources_writer.writerow([
                s.id,
                s.display_name,
                s.host_organization_name,
                s.works_count,
                s.cited_by_count,
                s.first_publication_year,
                s.last_publication_year,
                s.created_date,
                s.updated_date
            ])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid source record:", e)
            continue
sources_file.close()

In [34]:
for page in works_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            raw['related_works'] = [rel.replace(oa_replace, '') for rel in raw['related_works']]
            raw['corresponding_author_ids'] = [rel.replace(oa_replace, '') for rel in raw['corresponding_author_ids']]
            w = WorksModel(**raw)
            works_writer.writerow(
                [w.id,
                w.doi,
                w.title,
                w.publication_date,
                w.type,
                w.related_works,
                w.corresponding_author_ids]
            )
        except (ValidationError, KeyError) as e:
            print("Skipping invalid record:", e)
            continue
works_file.close()


In [35]:
for page in authors_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            a = AuthorsModel(**raw)
            authors_writer.writerow([
                a.id,
                a.display_name,
                a.works_count,
                a.cited_by_count,
                a.affiliations,
                a.topics,
                a.counts_by_year
            ])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid record:", e)
            continue
authors_file.close()

In [36]:
for page in topics_pager:
    for raw in page:
        raw['id'] = raw['id'].replace(oa_replace, '')
        raw['subfield'] = raw['subfield'].get('display_name', '')
        raw['field'] = raw['field'].get('display_name', '')
        raw['domain'] = raw['domain'].get('display_name', '')
        t = TopicsModel(**raw)
        topics_writer.writerow([
            t.id,
            t.display_name,
            t.description,
            t.subfield_name,
            t.field_name,
            t.domain_name,
            t.works_count,
            t.cited_by_count,
            t.updated_date,
            t.created_date,
        ])
topics_file.close()

In [37]:
for page in institutions_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            # Extract ROR ID from the full URL if present
            if raw.get('ror'):
                raw['ror'] = raw['ror'].replace('https://ror.org/', '')
            i = InstitutionsModel(**raw)
            institutions_writer.writerow([
                i.id,
                i.display_name,
                i.ror,
                i.country_code,
                i.type,
                i.works_count,
                i.cited_by_count,
                i.homepage_url,
                i.is_super_system,
                i.created_date,
                i.updated_date
            ])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid institution record:", e)
            continue
institutions_file.close()


In [38]:
for page in publishers_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            # Clean parent publisher ID if present - parent_publisher is a dict with 'id' key
            if raw.get('parent_publisher') and isinstance(raw['parent_publisher'], dict):
                parent_id = raw['parent_publisher'].get('id')
                if parent_id:
                    raw['parent_publisher'] = parent_id.replace(oa_replace, '')
                else:
                    raw['parent_publisher'] = None
            elif raw.get('parent_publisher') and isinstance(raw['parent_publisher'], str):
                raw['parent_publisher'] = raw['parent_publisher'].replace(oa_replace, '')
            
            p = PublishersModel(**raw)
            publishers_writer.writerow([
                p.id,
                p.display_name,
                p.country_code,
                # p.hierarchy_level,
                p.parent_publisher,
                p.works_count,
                p.cited_by_count,
                p.created_date,
                p.updated_date
            ])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid publisher record:", e)
            continue
publishers_file.close()


Skipping invalid publisher record: 1 validation error for PublishersModel
display_name
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
Skipping invalid publisher record: 1 validation error for PublishersModel
display_name
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
Skipping invalid publisher record: 1 validation error for PublishersModel
display_name
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [39]:
for page in funders_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            f = FundersModel(**raw)
            funders_writer.writerow([
                f.id,
                f.display_name,
                f.description,
                f.country_code,
                f.awards_count,
                f.works_count,
                # f.summary_stats,
                f.cited_by_count,
                f.homepage_url,
                f.created_date,
                f.updated_date
            ])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid funder record:", e)
            continue
funders_file.close()


#### Including Filter and Select

#### Relationship Creation attempt

In [40]:
class WorkAuthoredByEdge(BaseModel):
    work_id: str
    author_id: str
    author_position: str
    is_corresponding: bool

In [41]:
class WorkHasTopic(BaseModel):
    work_id: str
    topic_id: str
    score: Optional[float] = None

class WorkCitesWork(BaseModel):
    citing_work_id: str
    cited_work_id: str

class AuthorHasTopic(BaseModel):
    author_id: str
    topic_id: str
    score: Optional[float] = None

In [42]:
works_pager = Works().select(['id', 'authorships']).paginate(per_page=200)

In [43]:
works_pager_topics = Works().select(['id', 'topics']).paginate(per_page=200)

In [44]:
works_pager_citations = Works().select(['id', 'referenced_works']).paginate(per_page=200)

In [45]:
for page in works_pager:
    for raw in page:
        work_id = raw.get('id')
        if not work_id:
            continue
        
        work_id = work_id.replace(oa_replace, '')

        for auth in raw.get('authorships') or []:
            author_block = auth.get("author") or {}
            author_id = author_block.get("id")

            if not author_id:
                continue

            try:
                edge = WorkAuthoredByEdge(
                    work_id=work_id,
                    author_id=author_id.replace(oa_replace, ""),
                    author_position=auth.get("author_position"),
                    is_corresponding=auth.get("is_corresponding")
                )

                wa_edges_writer.writerow([
                    edge.work_id,
                    edge.author_id,
                    edge.author_position,
                    edge.is_corresponding
                ])

            except ValidationError:
                continue


wa_edge_file.close()

In [46]:
for page in works_pager_topics:
    for raw in page:
        work_id = raw.get('id')
        if not work_id:
            continue

        work_id = work_id.replace(oa_replace, '')

        for topic in raw['topics'] or []:
            topic_id = topic['id'].replace(oa_replace, '')
            topic_name = topic.get('display_name')
            topic_score = topic.get('score')

            wt_edges_writer.writerow([
                work_id,
                topic_id,
                topic_name,
                topic_score
            ])
wt_edge_file.close()            